# Set max_tokens to be close to the expected value

#### Load Helper Functions and Import Libraries

In [25]:
import datetime
import json
import time
import os
import datetime
import json
import time
from openai import AzureOpenAI
from dotenv import load_dotenv
import json
import copy
import textwrap

# cost per token for GPT 4 32K
prompt_token_cost = 0.06/1000
generation_token_cost = 0.12/1000


# Load environment variables
load_dotenv()

import os
from openai import AzureOpenAI

def aoai_call(system_message, prompt, model, max_tokens):
    client = AzureOpenAI(
        api_version=os.getenv("API_VERSION"),
        azure_endpoint=os.getenv("AZURE_ENDPOINT"),
        api_key=os.getenv("API_KEY")
    )

    start_time = time.time()

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
        max_tokens=max_tokens
    )

    end_time = time.time()
    e2e_time = end_time - start_time

    result = json.loads(completion.model_dump_json(indent=2))
    prompt_tokens = result["usage"]["prompt_tokens"]
    completion_tokens = result["usage"]["completion_tokens"]
    completion_text = result["choices"][0]["message"]["content"]

    return result, prompt_tokens, completion_tokens, completion_text, e2e_time
model=os.getenv("MODELGPT432k")

## Use case: Summarising a report

In [26]:
documents_to_check="""

1. “Quantum Entanglement: Spooky Action at a Distance”
Abstract:
Quantum entanglement, a phenomenon that baffled even Einstein, lies at the heart of quantum mechanics. In this article, we delve into the mysterious world of entangled particles, exploring how they can be connected across vast distances instantaneously. From Bell’s theorem to quantum teleportation, we unravel the enigma of entanglement and its potential applications in quantum computing and secure communication.

Introduction:
Quantum entanglement defies classical intuition. Imagine two particles—say, electrons—created together and then separated by light-years. Remarkably, their properties remain intertwined, regardless of the distance between them. When one particle’s state changes, the other responds instantaneously, as if they share a hidden connection. But how does this “spooky action at a distance” work?

Bell’s Theorem:
Physicist John Bell proposed a test to determine whether entanglement was real or merely a statistical fluke. Experiments confirmed Bell’s predictions: the correlations between entangled particles violated classical limits. Quantum mechanics prevailed, and entanglement emerged as a fundamental property of the universe.

Quantum Teleportation:
Entanglement enables quantum teleportation—a process where information about one particle is transmitted to another, even if they are light-years apart. This isn’t “Star Trek” teleportation of matter; instead, it transfers quantum states. Researchers are harnessing this phenomenon for secure communication and quantum networks.

Applications:
Beyond teleportation, entanglement plays a pivotal role in quantum computing. Qubits, the building blocks of quantum computers, rely on entanglement for their power. Scientists are also exploring entanglement-based sensors, clocks, and cryptography.

2. “CRISPR-Cas9: Rewriting the Genetic Code”
Abstract:
CRISPR-Cas9, a revolutionary gene-editing tool, has transformed biology and medicine. In this article, we explore the origins of CRISPR, its mechanism, and its impact on genetic research. From curing genetic diseases to creating designer organisms, CRISPR opens new frontiers in biotechnology.

Introduction:
Clustered Regularly Interspaced Short Palindromic Repeats (CRISPR) were initially discovered in bacteria as part of their immune system. Scientists soon realized that they could repurpose this system for precise gene editing. Enter CRISPR-Cas9—the Swiss Army knife of genetic manipulation.

How It Works:
CRISPR-Cas9 acts like molecular scissors. It uses a guide RNA to target specific DNA sequences, and the Cas9 protein cuts the DNA at that location. Researchers can then insert, delete, or modify genes with unprecedented accuracy. The simplicity and efficiency of CRISPR have revolutionized genetic research.

Applications:
Treating Genetic Diseases: CRISPR holds promise for curing genetic disorders like sickle cell anemia and cystic fibrosis. Clinical trials are underway.
Agriculture: CRISPR can create crops resistant to pests, drought, and disease.
Designer Babies?: Ethical debates surround using CRISPR for human enhancement.
Conservation: CRISPR may help save endangered species by editing their genomes.

"""

### A: Naive summarisation, using the model's default verbosity

In [27]:
system_message="""
You help write a summary of this document.
"""
prompt=f"""
Documents to summarize:
{documents_to_check}
"""

result,prompt_tokens,completion_tokens,completion_text,e2e_time=aoai_call(system_message,prompt,model,2000)
print(f"Prompt Tokens: {prompt_tokens}")
print(f"Completion Tokens: {completion_tokens}")
print(f"Total cost: ${prompt_tokens*prompt_token_cost + completion_tokens*generation_token_cost:.4f}")
print(f"Time taken: {e2e_time:.2f} seconds")
print(completion_text)


Prompt Tokens: 672
Completion Tokens: 170
Total cost: $0.0607
Time taken: 3.25 seconds
Title: Exploring Quantum Entanglement and CRISPR-Cas9

Summary:
The first document dives into the concept of quantum entanglement, a puzzling phenomenon that connects particles across vast distances instantly. It explores how physicist John Bell's theorem proved the reality of entanglement and how it enables quantum teleportation and plays a pivotal role in quantum computing, secure communication, and cryptography.

The second document delves into CRISPR-Cas9, a groundbreaking gene-editing tool derived from bacteria. It explains how CRISPR-Cas9 acts as molecular scissors to edit genes with high precision. The article highlights CRISPR's potential applications in treating genetic diseases, revolutionizing agriculture, potentially enabling human enhancement, and even aiding conservation efforts.

Both documents discuss the origins, mechanisms, and exciting potential of these cutting-edge scientific adv

### B: Set max_tokens to expected value (with margin)

In [28]:
estimated_max_tokens=300

system_message="""
You help write a summary of this document.
"""
prompt=f"""
Documents to summarize:
{documents_to_check}
"""

result,prompt_tokens,completion_tokens,completion_text,e2e_time=aoai_call(system_message,prompt,model,estimated_max_tokens)
print(f"Prompt Tokens: {prompt_tokens}")
print(f"Completion Tokens: {completion_tokens}")
print(f"Total cost: ${prompt_tokens*prompt_token_cost + completion_tokens*generation_token_cost:.4f}")
print(f"Time taken: {e2e_time:.2f} seconds")
print(completion_text)


Prompt Tokens: 672
Completion Tokens: 221
Total cost: $0.0668
Time taken: 4.71 seconds
Summary:

Document 1 titled "Quantum Entanglement: Spooky Action at a Distance" discusses the phenomenon of quantum entanglement and its applications. The article explains how entangled particles remain connected no matter the distance between them, and how this violates classical limits. It also explores how entanglement enables quantum teleportation, where information about one particle can be transmitted to another, even if they are far apart. The article mentions the potential applications of entanglement in quantum computing, secure communication, sensors, clocks, and cryptography.

Document 2 titled "CRISPR-Cas9: Rewriting the Genetic Code" focuses on the revolutionary gene-editing tool CRISPR-Cas9 and its impact on biology and medicine. The article explains how CRISPR-Cas9 acts like molecular scissors, targeting specific DNA sequences and allowing researchers to insert, delete or modify genes 